<a href="https://colab.research.google.com/github/Mat-O-Lab/CSVToCSVW/blob/main/csv_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-*- coding: UTF-8 -*-
#@title Code - Run Once To Start { vertical-output: true, display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
import base64
import json

%matplotlib notebook

!pip install Owlready2
from owlready2 import *

url = "https://raw.githubusercontent.com/Mat-O-Lab/CSVToCSVW/main/annotator.py"
file_name = 'csvtocsvw.py'

import os.path
if not os.path.isfile(file_name):
  urlretrieve(url,file_name) 
import csvtocsvw

#there is a bug in Owlready2 when having imports in turtle in a owl file
# if the error is thrown, load again and it is fine
try:
  mseo=get_ontology("https://purl.matolab.org/mseo/mid").load()
except:
  mseo=get_ontology("https://purl.matolab.org/mseo/mid").load()
  
cco_mu=get_ontology("http://www.ontologyrepository.com/CommonCoreOntologies/Mid/UnitsOfMeasureOntology/").load()
qudt=get_ontology('http://www.qudt.org/qudt/owl/1.0.0/unit.owl').load()

class AnnotatorDialog():
  def __init__(self, csv_url=''):
      self.csv_url = csv_url
  def _create_initial_widgets(self):
      self.url_widget=widgets.Text(
          value='',
          placeholder='put ur url to a *-metadata.json here',
          description='Url:',
          disabled=False
          )
      # self.uploader = widgets.FileUpload(accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
      #                                     multiple=False,  # True to accept multiple files upload else False
      #                                     description='Upload'
      #                                     )
      self.clear_button = widgets.Button(description='Clear!', layout=widgets.Layout(width='100px')); 
      self.file_dialog= widgets.HBox([widgets.Label(value="File:"), self.url_widget ])#,self.uploader,self.clear_button])
      self.clear_button.on_click(self._on_clear)
      
      self.out = widgets.Output()  # this is the output widget in which the df is displayed
      self.encoding = widgets.Dropdown(
          options=['auto', 'ISO-8859-1', 'utf-8', 'ascii', 'latin-1','cp273'],
          value='auto',
          description='Encoding:',
          disabled=False,
      )
      self.separator = widgets.Dropdown(
          options=['auto', ',',';', '\t', '|', "\s+","\s+|\t+|\s+\t+|\t+\s+"],
          value='auto',
          description='separator:',
          disabled=False,
      )
      self.settings= widgets.HBox([self.encoding, self.separator])
      self.process_button = widgets.Button(description='Process!', layout=widgets.Layout(width='200px')); 
      self.process_button.on_click(self._on_process)
  def _on_clear(self,button):
    self.url_widget.value=''
    self.uploader.value.clear()
    self.uploader._counter = 0

  def _on_process(self,button):
    with self.out:
      clear_output()
      if not (self.url_widget.value or self.uploader.value.keys()):
          print('pls upload a file first or insert a url')
          return
      if self.url_widget.value:
        self.data_uri=self.url_widget.value
      else:
        print('no input uri given')
      self.annotator=csvtocsvw.CSV_Annotator(separator=self.separator.value,encoding=self.encoding.value)
      metafile_name, result=self.annotator.process(self.data_uri)
      print(result)
      res = result
      b64 = base64.b64encode(res.encode())
      payload = b64.decode()
      html_buttons = '''<html>
      <head>
      <meta name="viewport" content="width=device-width, initial-scale=1">
      </head>
      <body>
      <a download="{filename}" href="data:text/json;base64,{payload}" download>
      <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
      </a>
      </body>
      </html>
      '''
      html_button = html_buttons.format(payload=payload,filename=metafile_name)
      display(widgets.HTML(html_button))

    
  def display_widgets(self):
    self._create_initial_widgets()
    display(widgets.VBox(
                [
                  self.file_dialog,
                  self.settings,
                  self.process_button,
                  self.out
                ]
            )
    )

In [ ]:
#@title Dialog - Run Cell to begin
dialog = AnnotatorDialog()
dialog.display_widgets()